In [ ]:
%pip install delta-spark==2.3.0
%pip install soda-core-spark-df
%pip install -U pandas==1.5.3

In [ ]:
from packages.business_logic.dq_etl_workflow import DQETLWorkflow

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os
import shutil


spark_conf = SparkConf()
spark_conf.set("spark.executor.memory", "6g")
spark_conf.set("spark.driver.memory", "6g")
spark_conf.set("spark.network.timeout", "600s")
spark_conf.set("spark.executor.instances", "4")
spark_conf.set("spark.executor.cores", "4")
spark_conf.set("spark.default.parallelism", "6")
spark_conf.set("spark.sql.shuffle.partitions", "6")
spark_conf.set("spark.sql.parquet.enableVectorizedReader", "true")

spark = SparkSession.builder.enableHiveSupport().config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
.config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.config(conf=spark_conf).getOrCreate()

In [ ]:
def delete_directories():
    for layer in ["analytics", "raw", "trusted"]:
        for root, dirs, files in os.walk(os.path.join(os.getcwd(), "layers", layer)):
            for file in files:
                os.remove(os.path.join(root, file))
            for dir in dirs:
                shutil.rmtree(os.path.join(root, dir))
            shutil.rmtree(os.path.join(os.getcwd(), "layers", layer))
delete_directories()

In [ ]:
dq_etl_workflow = DQETLWorkflow(spark_session=spark, include_dq_workflow=False)
dq_etl_workflow.execute()